### Embedded Systems Final Project - Code: 

In [ ]:
import cv2
import numpy as np
from jetbot import Robot, Camera, bgr8_to_jpeg
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
from jetbot import Robot
from IPython.display import clear_output

In [ ]:
robot = Robot()

In [ ]:
#Region of interest (ROI) in the image. The ROI is defined by the variables roi_left, roi_top, roi_right, and roi_bottom.
roi_left = 30
roi_right = 194
roi_top = 20
roi_bottom = 194
roi_mid = int((roi_right-roi_left)/2) # The roi_mid variable is the midpoint of the ROI, calculated as the average of roi_left and roi_right.

# Range values (lower-upper) The variables TLow and THigh are the lower and upper range values for threshold operation.
TLow = 0
THigh = 50

TShow = 0 # The TShow variable is a boolean that determines whether or not to show image 

TThreshold = 80

# Error Tolerance
Error_dev = 2

# The nav_angle variable is a global variable that is used to store the navigation angle of a system. It is initially set to 0.
global nav_angle
nav_angle = 0



#### Black Line Detection:

In [ ]:
def BLine(img, TShow, TLow, THigh):
    default = img.copy()
    cv2.line(default, (roi_left, roi_top), (roi_right, roi_top), (0, 0, 255), 1)
    cv2.line(default, (roi_left, roi_bottom), (roi_right, roi_bottom), (0, 0, 255), 1)
    cv2.line(default, (roi_mid, roi_top), (roi_mid, roi_bottom), (0, 255, 0), 2)


    img = img[roi_top:roi_bottom, roi_left:roi_right]
    frame = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    frame = cv2.GaussianBlur(frame, (11,11), 0)
    _, frame = cv2.threshold(frame, TThreshold, 255, cv2.THRESH_BINARY_INV)
    frame = cv2.inRange(frame, TLow, THigh)
    frame = cv2.erode(frame, None, iterations=3)
    frame = cv2.dilate(frame, None, iterations=3)
    frame = cv2.bitwise_not(frame)
    
    contours_blk, hierarchy_blk = cv2.findContours(frame, 1, cv2.CHAIN_APPROX_SIMPLE) 
# contours_blk variable is a list of contours detected in an image and the cv2.findContours function is used to find the contours in the image.
# The cv2.findContours function returns two values: the first is a list of contours and the second is a hierarchy of contours. 
# The hierarchy is not used in this code, so it is stored in the hierarchy_blk variable. The contours_blk variable stores the list of contours.

    if len(contours_blk) == 0:
        cs = None

    if len(contours_blk) > 0:
        c = max(contours_blk, key=cv2.contourArea)
        M = cv2.moments(c)
        try:
            cx = int(M['m10']/M['m00'])
            cy = int(M['m01']/M['m00'])+roi_top
            error = cx - roi_mid
        except:
            cx = None
            print("Error!")
        
        if cx != None:
            if TShow == 0:
                cv2.drawContours(img, contours_blk, -1, (0, 255, 0), 1)
                cv2.line(default, (cx, roi_top), (cx, roi_bottom), (255, 0, 0), 1)
                cv2.line(default, (roi_left, cy), (roi_right, cy), (255, 0, 0), 1)
        
            cv2.circle(default, (cx, cy), 5, (0,255,0), 3)
            cv2.line(default, (cx,cy), (roi_mid, cy), (0, 0, 255), 2)
          
    else:
        cx = None


# If there are any contours in the list, the code selects the contour with the largest area using the max function and a key function that returns the area of a contour. 
# The selected contour is stored in the c variable. The cv2.moments function is then used to calculate the moments of the selected contour. Moments are used to describe the shape of an object in an image. 
# The cv2.moments function returns a dictionary of moments calculated for the contour. The M variable stores this dictionary.
# The code then calculates the centroid of the contour by dividing the m10 and m01 moments by the m00 moment. The centroid is the center of the contour and is represented by the cx and cy variables. The cx variable is the x-coordinate of the centroid, and the cy variable is the y-coordinate of the centroid.

    return frame, img, default, cx

# This function processes an image to detect a black line in it. The input to the function is an image img, a boolean TShow, and two integers TLow and THigh. The function first creates a copy of the image called default and draws two horizontal lines on it to mark the top and bottom boundaries of a ROI within the image and also draws a vertical line to mark the middle of the ROI. 
# The function then selects the ROI from the original image img by cropping it to the region between the top and bottom lines and between the left and right edges of the image.
# The function then converts the cropped image to grayscale using the cv2.cvtColor function and applies Gaussian blur to it using the cv2.GaussianBlur function. It then applies binary thresholding to the blurred image using the cv2.threshold function, inverts the resulting binary image using the cv2.bitwise_not function, and applies erosion and dilation to it using the cv2.erode and cv2.dilate functions, respectively. 
# All of these are done to isolate and identify the black line in the image. The function then finds the contours of the resulting image using the cv2.findContours function.
# Using that, if the function detects a black line in the image, it calculates the centroid of the contour and the error between the centroid and the middle of the ROI. It then draws the contour, the centroid, and the error on the images img and default. If the TShow input is set to True, the function also displays the original image with the black line contours drawn on it and the default image with the lines, centroid, and error drawn on it.
# The function returns the binary image with the black line contours, the original image with the black line contours drawn on it, the default image , and the x-coordinate of the centroid. If no black line is detected, the function returns None for the x-coordinate of the centroid/center of intersection.

#### Black Line Green Signal Detection:

In [ ]:
def BLine_GSignal(img, TShow, TLow, THigh):
    mode = "center"
    GDetect = False
    kernel = np.ones((3,3), np.uint8)
    default = img.copy()
    cv2.line(default, (roi_left, roi_top), (roi_right, roi_top), (0, 0, 255), 1)
    cv2.line(default, (roi_left, roi_bottom), (roi_right, roi_bottom), (0, 0, 255), 1)
    cv2.line(default, (roi_mid, roi_top), (roi_mid, roi_bottom), (0, 255, 0), 2)


    img = img[roi_top:roi_bottom, roi_left:roi_right]
    
    # Black Line Image Processing
    frame = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    frame = cv2.GaussianBlur(frame, (11,11), 0)
    _, frame = cv2.threshold(frame, TThreshold, 255, cv2.THRESH_BINARY_INV)
    frame = cv2.inRange(frame, TLow, THigh)
    frame = cv2.erode(frame, None, iterations=3)
    frame = cv2.dilate(frame, None, iterations=3)
    frame = cv2.bitwise_not(frame)
    contours_blk, hierarchy_blk = cv2.findContours(frame, 1, cv2.CHAIN_APPROX_SIMPLE) #find contours with simple approximation similar to above
    
    # Green Signal Image Processing
    GreenSignal = cv2.inRange(img, (0,65,0), (100,200,100))
    GreenSignal = cv2.erode(GreenSignal, kernel,iterations=5)
    GreenSignal = cv2.dilate(GreenSignal, kernel,iterations=9)
    GreenSignal = cv2.bitwise_not(GreenSignal)
    
    contours_grn, hierarchy_grn = cv2.findContours(GreenSignal.copy(), cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE) #find contours with simple approximation similar to above

# If there are any contours in the list, the code sets the GDetect variable to True and calculates the bounding rectangle of the first contour in the list using the cv2.boundingRect function. 
# The bounding rectangle is a rectangle that surrounds the contour and is represented by the x_grn, y_grn, w_grn, and h_grn variables. The x_grn and y_grn variables represent the top-left corner of the bounding rectangle, while the w_grn and h_grn variables represent the width and height of the bounding rectangle, respectively.

# The code then calculates the center of the bounding rectangle by adding the x_grn and y_grn variables to half of the w_grn and h_grn variables, respectively. The center of the bounding rectangle is represented by the centerx_grn and centery_grn variables.
# If the x_grn or y_grn or w_grn or h_grn variables are None, the code sets the GDetect variable to False.
    
    if len(contours_grn) > 0 :
        GDetect = True
        x_grn, y_grn , w_grn, h_grn = cv2.boundingRect(contours_grn[0])
        if(x_grn is None or  y_grn is None or w_grn or h_grn is None):
            GDetect = False
        centerx_grn = x_grn + (w_grn/2)
        centery_grn = y_grn + (h_grn/2)
        
       
       # print("Green center y", centery_grn)

    if len(contours_blk) > 0:
        c = max(contours_blk, key=cv2.contourArea)
        M = cv2.moments(c)
        try:
            cx = int(M['m10']/M['m00'])
            cy = int(M['m01']/M['m00'])+roi_top
            error = cx-roi_mid
        except:
            #cx = None
            cx = roi_mid
            cy = roi_mid
            print("Error!")

    else:
        cx = roi_mid
        cy = roi_mid
        
    if GDetect: 
        deviation = abs(centerx_grn - cx)
        if centerx_grn > cx and centery_grn > 150 and deviation<15 and deviation > 0:
        #if centerx_grn > cx and centery_grn > 125:

            mode = "right"
            cx = centerx_grn + 100
            #print("Turn right")
            # cv2.putText(img, "Turn Right", (100,100), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0),1)

        elif centerx_grn < cx and centery_grn > 150 and deviation<15 and deviation > 0:
        #elif centerx_grn <= cx and centery_grn > 125:
            mode = "left"
            #print("Turn left")
            cx = centerx_grn - 100
            # cv2.putText(img, "Turn Left", (100,100), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0),1)
            
            
    return frame, img, default, cx, mode



#Explanation:
# This function  processes an image from the robot's camera to detect the position of a black line and a green signal within a region of interest (ROI) in the image defined by the variables roi_left, roi_top, roi_right, and roi_bottom.
#The function first creates a copy of the image called default and draws two horizontal lines on it to mark the top and bottom boundaries of a region of interest (ROI) within the image. It also draws a vertical line to mark the middle of the ROI. The function then selects the ROI from the original image img by cropping it to the region between the top and bottom red lines and between the left and right edges of the image.
# The function then processes the image to detect the black line and the green signal. First, the image is converted to grayscale and then blurried using a Gaussian blur. The image is then thresholded using the cv2.threshold() function to create a binary image. The binary image is then eroded and dilated to reduce noise and make the contours of the objects in the image more distinct.
# The function uses the cv2.findContours() function to find the contours of the black line and the green signal in the image. The contours of the black line are stored in the contours_blk variable, and the contours of the green signal are stored in the contours_grn variable.
# The function then checks if the green signal was detected in the image by checking the length of the contours_grn list. If the green signal was detected, the function sets the GDetect flag to True and calculates the center position of the green signal using the x_grn, y_grn, w_grn, and h_grn variables, which are the bounding box coordinates of the green signal.
# The function then checks if the black line was detected in the image by checking the length of the contours_blk list. If the black line was detected, the function calculates the center position of the black line using the moments of the largest contour in the contours_blk list. The center position is stored in the variables cx and cy.
# Finally, the function determines the mode of the image based on the position of the black line and the green signal. If the green signal is detected and is to the right of the black line, the function sets the mode to "right". If the green signal is not detected, or if it is to the left of the black line, the function sets the mode to "center", and the function returns the binary image of the black line, the original image with the ROI marked, the default image with the ROI marked, the center position of the black line, and the mode of the image.

#### Camera Output/View:

In [ ]:
camera = Camera.instance()
image_widget = widgets.Image()
traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

display(image_widget)

# Explanation:
#
# This code block uses the traitlets module to link the value attribute of a Camera object to the value attribute of an Image widget. The Camera object is used to capture images from a camera, and the Image widget is used to display the images.
# The Camera.instance() function is used to create a singleton instance of the Camera class. We use this because a singleton allows only one instance of a class to be created. This is useful in situations where you want to ensure that there is only one object of a certain class that is shared among all users of that object such as that in this case of the camer and the image from it.
# The bgr8_to_jpeg function is used to convert the value attribute of the Camera object, which is in the BGR8 format, to JPEG format before it is displayed in the Image widget. The display function is then used to show the Image widget.
# When the code is run, the Image widget will display the images captured by the Camera object in real-time.

In [ ]:
image = camera.value.copy()
frame, img, oryginal, cx, mode = BLine_GSignal(image, TShow, TLow, THigh)
print(roi_mid, cx)

print(mode)
if(cx>=roi_mid + Error_dev):
    print("Right!")
elif(cx<=roi_mid - Error_dev):
    print("Left!")
else:
    print("Center!")

#Explanation:
# This code block captures an image from a camera using the Camera object, processes the image to detect a black line and a green signal in it using the BLine_GSignal function, and then checks the position of the black line relative to the center of the region of interest (ROI).
# The BLine_GSignal function is called with the image, TShow, TLow, and THigh arguments and returns five values: frame, img, oryginal, cx, and mode. The frame and img variables are the binary images with the black line contours, the original variable is the original image with the black line contours drawn on it, the cx variable is the x-coordinate of the centroid of the black line, and the mode variable is a string indicating the mode of the system (e.g., "center", "left", "right").
# The roi_mid variable is the midpoint of the ROI and is calculated as the average of roi_left and roi_right. The Error_dev variable is an error tolerance value.

# After the image is processed, the code compares the value of cx to roi_mid with an error tolerance of Error_dev. If cx is greater than roi_mid + Error_dev, the code prints "Right!". If cx is less than roi_mid - Error_dev, the code prints "Left!". If cx is within the error tolerance range, the code prints "Center!".

In [ ]:
speed_gain_slider = widgets.FloatSlider(min=0.0, max=1.0, step=0.01, value = 0.15, description='speed gain')
steering_gain_slider = widgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.30, description='steering gain')
steering_dgain_slider = widgets.FloatSlider(min=0.0, max=0.5, step=0.001, value=0.15, description='steering kd')
steering_bias_slider = widgets.FloatSlider(min=-0.3, max=0.3, step=0.01, value=0.0, description='steering bias')

display(speed_gain_slider, steering_gain_slider, steering_dgain_slider, steering_bias_slider)

# This code creates four sliders using the widgets module. 
# The first slider is a float slider with a minimum value of 0.0, a maximum value of 1.0, a step size of 0.01, and an initial value of 0.15.
# The second slider is a float slider with a minimum value of 0.0, a maximum value of 1.0, a step size of 0.01, and an initial value of 0.30. 
# The third slider is a float slider with a minimum value of 0.0, a maximum value of 0.5, a step size of 0.001, and an initial value of 0.15.
# The fourth slider is a float slider with a minimum value of -0.3, a maximum value of 0.3, a step size of 0.01, and an initial value of 0.0. 
# The display function is then used to show the sliders. These sliders can be used to adjust the values of four variables (the speed gain, steering gain, steering kd, and steering bias, respectively) in real-time by moving the sliders.

#### Navigation Block:

In [ ]:

#PID and navigation resource #https://www.youtube.com/watch?v=4Y7zG48uHRo

def Navigation(cx, roi_mid, nav_angle):
    range_val = roi_left - roi_right
    angle = (roi_mid - cx) / range_val
    
    #pid calculation
    pid = angle * steering_gain_slider.value + (angle - nav_angle) * steering_dgain_slider.value
    nav_angle = angle
    
    steering = pid + steering_bias_slider.value

    
    left =  max(min(speed_gain_slider.value + steering, 1.0), 0.0)
    right = max(min(speed_gain_slider.value - steering, 1.0), 0.0)
    # print(left,right)
    # clear_output(wait = True)

    robot.left_motor.value = max(min(speed_gain_slider.value + steering, 0.5), 0.0)
    robot.right_motor.value = max(min(speed_gain_slider.value - steering, 0.5), 0.0)
    
#   Explanation:
#
#   The Navigation function is responsible for controlling the car's movement based on the input values of cx, roi_mid, and nav_angle. 
#   These values are used to calculate the angle of the car relative to the line being followed and the desired speed of the car.
#   The function begins by calculating the range of values between roi_left and roi_right and storing the result in the variable range_val. 
#   It then calculates the angle of the car relative to the line being followed by dividing the difference between roi_mid and cx by range_val and storing the result in the variable angle.
#   Next, the function calculates the pid value by multiplying angle by the value of the steering_gain_slider widget and adding the product of the difference between angle and nav_angle and the value of the steering_dgain_slider widget.
#   The function then adds the value of the steering_bias_slider widget to the pid value and stores the result in the variable steering.
#   Finally, the function uses the steering value and the speed_gain_slider value to calculate the speed of the left and right motors of the car. The calculated speeds are then clamped to the range between 0 and 0.5 and applied to the motors using the robot object. 


#### Execution Block:

In [ ]:
cx_prev = 0
TShow = False
while True:
    # print(mode)
    image = camera.value.copy() #image from camera is copied to variable image
    frame, img, default, cx, mode = BLine_GSignal(image, TShow, TLow, THigh)
#setting steering and speed to 0 if no black line is detected
    if(cx is None):
        cx = roi_mid
        steering_gain_slider*=0.0
        speed_gain_slider*=0.0

#setting steering and speed gain sliders as per mode
    if(mode=="right" or mode=="left"):
        steering_gain_slider.value*=2.0
        speed_gain_slider.value*=1.5
        for i in range(2500):
            Navigation(cx, roi_mid,0)
        steering_gain_slider.value/=2.0
        speed_gain_slider.value/=1.5
    else:
        #if(abs(cx-roi_mid)>=50):
            #cx = roi_mid                                                                                                                                                                                                   
        Navigation(cx, roi_mid,0)
   
    # if TShow:
    #     display(JupyterImage(img))
    # else:
    #     display(JupyterImage(frame))
    # cx_prev = cx

#   Explanation:
#
#   This code  block is responsible for calling the Navigation function with the appropriate arguments and using the BLines_GSignal function to get the values of cx and mode, resposinble for controlling the robot and start its movement on the track
#   If the value of mode is "right" or "left", then the code multiplies the values of steering_gain_slider and speed_gain_slider by 2.0 and 1.5, respectively. It then calls the Navigation function 2500 times with the values of cx, roi_mid, and 0 as arguments. After the loop, it divides the values of steering_gain_slider and speed_gain_slider by 2.0 and 1.5, respectively.
#   If the value of mode is not "right" or "left", then the code calls the Navigation function with the values of cx, roi_mid, and 0 as arguments.

In [ ]:

# robot.left_motor.value = 0
# robot.right_motor.value = 0
robot.stop()

camera.close()

#  Explanation:
#  This code block is responsible for stopping the robot and closing the camera object.